In [1]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [2]:
from project_heart.examples import get_lv_typeA
lv = get_lv_typeA(filepath="C:/Users/igorp/Downloads/0.28_70.00_40.00_LVMYO_HEX8_83648_wr.xplt", statesfile=None)

INFO:LV_RegionIdentifier.identify_regions_typeA:Identifying regions from 'type A' geometry.
INFO:LV_RegionIdentifier.identify_endo_epi_base_borders_from_nodesets:len(endo_base): 487
INFO:LV_RegionIdentifier.identify_endo_epi_base_borders_from_nodesets:len(epi_base): 507


In [3]:
# spks = lv.get_speckles(spk_collection="circ-6", spk_group="endo")

In [4]:
# names = ["subapex", "apex", "superapex", "submid", "mid", "supermid", "subbase", "base", "superbase"]
k = 0.89
d = 1.5
spks1 = lv.create_speckles(
            collection="MANUAL_CALC_CIRC_6",
            group="endo",
            name="superbase",
            from_nodeset=LV_SURFS.ENDO,
            d=d,
            k=k,
            normal_to=[0.0, 0.0, 1.0],
            n_subsets=6,
            subsets_criteria="angles",
            cluster_criteria="angles3",
            n_clusters=16,
            t=0.0,
        )
_ = lv.create_speckles(
            collection="MANUAL_CALC_CIRC_6",
            group="epi",
            name="superbase",
            from_nodeset=LV_SURFS.EPI,
            d=d,
            k=k,
            normal_to=[0.0, 0.0, 1.0],
            n_subsets=6,
            subsets_criteria="angles",
            cluster_criteria="angles3",
            n_clusters=16,
            t=0.0,
        )

In [5]:
# spks = lv.get_speckles(spk_collection="MANUAL_CALC_CIRC_6")
# lv.plot_speckles(spks, show_longitudinal_line=True, show_clusters_centers=True)

In [6]:
_ = lv.create_speckles(
            collection="MANUAL_CALC_LONG_1",
            group="endo",
            name="0.0",
            from_nodeset=LV_SURFS.ENDO,
            exclude_nodeset=LV_SURFS.BASE, # does not afect ideal case
            d=1.00,
            k=0.5,
            normal_to=[1.0, 0.0, 0.0],
            n_subsets=0,
            subsets_criteria="z2",
            cluster_criteria="z2",
            n_clusters=30,
            t=0.0,
        )

In [7]:
# spks = lv.get_speckles(spk_collection="MANUAL_CALC_LONG_1")
# lv.plot_speckles(spks, show_longitudinal_line=True, show_clusters_centers=True)

# Metrics

In [8]:
df = lv.extract_geometrics({
        "volume":{},
        "apex_base_over_timesteps": {
            "apex_spks": {"spk_name":"apex", "spk_group":"endo", "spk_collection":"LA"},
            "base_spks": {"spk_name":"base", "spk_group":"endo", "spk_collection":"LA"},
            },
        "longitudinal_distance": {
            "apex_spks": {"spk_name":"apex", "spk_collection":"LA"},
            "base_spks": {"spk_name":"base", "spk_collection":"LA"},
            "approach":"centroid", 
            "use_axis_aligment":False,
            "reduce_by":{"group"}
            },
        "radial_distance": {
            "spks": {"spk_collection":"MANUAL_CALC_CIRC_6"},
            "approach":"moving_vector",
            "reduce_by":{"group", "name", "group_name"},
            },
        "radial_length": {
            "spks": {"spk_collection":"MANUAL_CALC_CIRC_6"},
            "approach":"moving_centers",
            "reduce_by":{"group", "name", "group_name"},
            },
        "wall_thickness": {
            "endo_spks": {"spk_group":"endo", "spk_collection":"MANUAL_CALC_CIRC_6"},
            "epi_spks": {"spk_group":"epi", "spk_collection":"MANUAL_CALC_CIRC_6"},
            "approach":"radial_distance",
            "reduce_by":{"name"},
            },
        "longitudinal_length": {
            "spks": {"spk_collection":"long-6"},
            "approach":"k_ids",
            "reduce_by":{"subset"},
            "merge_subset":{
                    5: 0, 
                    4: 1, 
                    3: 2
                },
            },
        "global_longitudinal_length": {
            "spks": {"spk_collection":"MANUAL_CALC_LONG_1"},
            "approach":"k_ids",
            "reduce_by":{"group", "group_name"},
            },
        "circumferential_length": {
            "spks": {"spk_collection":"MANUAL_CALC_CIRC_6"},
            "reduce_by":{"group"},
            },
        "global_circumferential_length": {
            "spks": {"spk_collection":"circ-1"},
            "reduce_by":{"group", "group_name"},
            },
        "angle_rotation": {
            "spks": {"spk_collection":"MANUAL_CALC_CIRC_6"},
            "reduce_by":{"group", "name"},
            }
        }
    )

INFO:LV:Computing apex_base_over_timesteps.
INFO:LV.BaseMetricsComputations.compute_base_apex_ref_over_timesteps:Computing apex and base virtual nodes over timesteps
INFO:LV:Extracting volume.
INFO:LV:Extracting longitudinal_distance.
INFO:LV.BaseMetricsComputations:Computing metric 'LV_STATES.LONGITUDINAL_DISTANCE'.
INFO:LV.BaseMetricsComputations:Computing 'LV_STATES.LONGITUDINAL_DISTANCE' with approach 'centroid' with axis aligment set to 'False'.
INFO:LV.BaseMetricsComputations:Computing 'LV_STATES.LONGITUDINAL_DISTANCE' with approach 'centroid' with axis aligment set to 'False'.
INFO:LV:Extracting radial_distance.
INFO:LV.BaseMetricsComputations:Computing metric 'LV_STATES.RADIAL_DISTANCE'
INFO:LV:Extracting radial_length.
INFO:LV.BaseMetricsComputations:Computing metric 'LV_STATES.RADIAL_LENGTH'
INFO:LV:Extracting wall_thickness.
INFO:LV.BaseMetricsComputations:Computing metric 'LV_STATES.WALL_THICKNESS'
INFO:LV:Extracting longitudinal_length.
INFO:LV.BaseMetricsComputations:Comp

In [9]:
dfoi = df[[
    "timesteps",
    lv.STATES.LONGITUDINAL_DISTANCE_ENDO.value, 
    lv.STATES.RADIAL_DISTANCE_ENDO_SUPERBASE.value, 
    lv.STATES.RADIAL_DISTANCE_EPI_SUPERBASE.value,
    lv.STATES.WALL_THICKNESS_SUPERBASE.value, 
    "global_longitudinal_length_endo_0.0", 
    lv.STATES.GLOBAL_CIRCUMFERENTIAL_LENGTH_ENDO_SUPERBASE.value,     
    ]]

In [11]:
dfoi.to_csv("test_0.28_70.00_40.00_LVMYO_HEX8_83648_wr.csv")

In [ ]:
# spks = lv.get_speckles(spk_collection="circ-1")
# lv.plot_speckles(spks,  show_clusters=True, show_clusters_centers=True)

In [12]:
spks = lv.get_speckles(spk_collection="MANUAL_CALC_LONG_1")
lv.plot_speckles(spks, show_longitudinal_line=True, show_clusters_centers=True)

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, 1…